In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pandas as pd
from os import getcwd
from os.path import join
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../data/gifts.csv')

In [ ]:
df

In [ ]:
x1 = df['Latitude'].to_numpy(np.float64)
x2 = df['Longitude'].to_numpy(np.float64)

## Creating K-means cluster
find out optimal number of clusters

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt

x1 = df['Latitude'].to_numpy(np.float64)
x2 = df['Longitude'].to_numpy(np.float64)

plt.plot()
plt.xlim([0, 10])
plt.ylim([0, 10])
plt.title('Dataset')
plt.scatter(x1, x2)
plt.show()

# create new plot and data
plt.plot()
X = np.array(list(zip(x1, x2))).reshape(len(x1), 2)
colors = ['b', 'g', 'r']
markers = ['o', 'v', 's']

# k means determine k
distortions = []
K = range(1,100)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(X)
    kmeanModel.fit(X)
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import seaborn as sns

df_coordinates=df[['Latitude','Longitude']]

m = KMeans(n_clusters=20, n_init=3, max_iter=3000, random_state=1)
#m = KMeans(20)
m.fit(df_coordinates)

df_coordinates['cl'] = m.labels_
df_coordinates.sort_values(by=['cl'], inplace=True)

fig, ax = plt.subplots(figsize=(20,8))
sns.scatterplot(data=df_coordinates, x="Longitude", y="Latitude", hue="cl", size=2)

In [ ]:
df_coordinates.plot.scatter('Longitude', 'Latitude', c='cl', colormap='gist_rainbow')

# Dataframe statistics

In [ ]:
df.describe()

In [ ]:

print("Number of rows : ", len(df))
print(" ")
print("Min Weight: ", df.Weight.min())
print("Max Weight: ", df.Weight.max())

print("Min Longitude: ",df.Longitude.min())
print("Max Longitude: ",df.Longitude.max())
print("Min Latitude: ",df.Latitude.min())
print("Max Latitude: ",df.Latitude.max())

plt.rcParams['figure.figsize'] = 20, 10
plt.hist2d(df.Longitude, df.Latitude, bins=360)
cb = plt.colorbar()
cb.set_label("density")

In [ ]:
plt.hist(df['Weight'], bins = 30)

# Harvesine Distance

- extract lat & lon
- stack values into np.array
- reshape np.array to (-1,2)
- convert lat & lon to radians
- compute pairwise harvesine distance using sklearn


## Basic examples (Sklearn)

### 1. using lists

In [ ]:
from sklearn.metrics.pairwise import haversine_distances
from math import radians

bsas = [-34.83333, -58.5166646]
paris = [49.0083899664, 2.53844117956]
london = [51.5074, 0.1278]

bsas_in_radians = [radians(_) for _ in bsas]
paris_in_radians = [radians(_) for _ in paris]
london_in_radians = [radians(_) for _ in london]

print(bsas_in_radians)
print(paris_in_radians)
print(london_in_radians)
result = haversine_distances([bsas_in_radians, paris_in_radians,london_in_radians])
result * 6371000/1000  # multiply by Earth radius to get kilometers

### 2. using np.arrays

In [ ]:
bsas = [-34.83333, -58.5166646]
paris = [49.0083899664, 2.53844117956]
london = [51.5074, 0.1278]

cities = np.array([bsas,paris,london])
cities_radian = np.radians(cities)
result = haversine_distances(cities_radian)
result * 6371000/1000  # multiply by Earth radius to get kilometers

### Pandas and Numpy

In [ ]:
# The first number is always the latitude and the second is the longitude ;)
coords = df[['Latitude','Longitude']].head(10).to_numpy()
coords = np.radians(coords)
result = haversine_distances(coords)
result * 6371000/1000  # multiply by Earth radius to get kilomet
#df['Latitude'] = coords[:,0]
#df['Longitude'] = coords[:,1]

## Why Not to calculate pairwise distance between all locations..
- The adj. Matrix of the complet craph would need n**2 * np.float memory
- As calculated below, this would not be wise :D but for each tour it would be no problem (mean weight = 14.xx --> 70 stops on a maximum capacity of 1000) 
- Nevertheless, it could be useful to just transform the coordinates into radians anyway. That way, it wouldn't be necessary to do the calculations each time a sample is used for the weighted reindeer wearniess...

In [ ]:
adj_mat_size = len(df)
print("Adj Matrix dimension: (" ,adj_mat_size,",",adj_mat_size,")")
print("Memory needed (GB):", (adj_mat_size**2)*8/10**9 ) # roughly 8 bytes for a float in numpy arrays

# Weighted reindeer weariness

$$WRW = \sum\limits_{j=1}^{m} \sum\limits_{i=1}^{n} \Big[ \big( \sum\limits_{k=1}^{n} w_{kj} - \sum\limits_{k=1}^{i} w_{kj} \big) \cdot Dist(Loc_i, Loc_{i-1}) \Big]_j ,$$

$$m := \text{number of trips} $$
$$j := \text{one specific trip}$$
$$n := \text{nmber of gifts (per trip j) }$$
$$w_{ij} := \text{weight of the }i^{th} \text{ gift at trip j}$$ 
$$Loc_{0}\text{ and } Loc_{0} \text{is the north pole for each trip j}$$ 
$$w_{nj} := \text{is the weight of the empty sled}$$ 

## Mini Example
Example with the first ten entries:
- trip 1 = entries 0:4
- trip 2 = entries 5:9

trip1 : North_Pole --> 0 --> 1 --> 2 --> 3 --> 4 --> North_Pole\
trip2 : North_Pole --> 5 --> 6 --> 7 --> 8 --> 9 --> North_Pole

In [ ]:
def weighted_reindeer_weariness(trips):
    weighted_weariness = 0
    for trip in trips:
        weights = trip['Weight'].to_numpy()
        coordinates = trip[['Latitude','Longitude']].to_numpy()
        weighted_weariness = weighted_weariness + weighted_distance(coordinates,weights,sleigh_weight)
    return weighted_weariness
    
def weighted_distance(coordinates,weights,sleigh_weight):
    startweight = sleigh_weight + np.sum(weights)
    if startweight > weight_limit:
        return -1

    north_pole = np.radians([90,0])
    coords = np.vstack((north_pole,coordinates,north_pole))
  
    adj_matrix = haversine_distances(coords,np.roll(coords.copy(),-1,axis=0))
    adj_matrix = adj_matrix * 6371 #6371000/1000
    distances = np.diag(adj_matrix)[:-1]
    
    weights +=sleigh_weight
    weights = np.append(weights,sleigh_weight)
    weights = np.cumsum(weights[::-1])[::-1] # flip, cummulative sum, flip again

    """
    print(coords)
    for i in range(len(coords)-1):
        print(haversine_distances([coords[i],coords[i+1]])[0][1]*6371)
    
    with np.printoptions(precision=3, suppress=True):
        print(distances,2)
    """
    weighted_dist = np.sum(weights*distances)
    print("weighted_dist ",weighted_dist)


    return weighted_dist


In [ ]:
def weighted_reindeer_weariness(trips):
    weighted_weariness = 0
    for trip in trips:
        weights = trip['Weight'].to_numpy()
        coordinates = trip[['Latitude','Longitude']].to_numpy()
        weighted_weariness = weighted_weariness + weighted_distance(coordinates,weights,sleigh_weight)
    return weighted_weariness
    
def weighted_distance(coordinates,weights,sleigh_weight):
    startweight = sleigh_weight + np.sum(weights)
    if startweight > weight_limit:
        return -1

    north_pole = np.radians([90,0])
    coords = np.vstack((north_pole,coordinates,north_pole))
    
    print(coords)
    distances = []
    for i in range(len(coords)-1):
        distances.append(haversine_distances([coords[i],coords[i+1]])[0][1]*6371)
    distances = np.array(distances)
    #adj_matrix = haversine_distances(coords,np.roll(coords.copy(),-1,axis=0))
    #adj_matrix = adj_matrix * 6371 #6371000/1000
    #distances = np.diag(adj_matrix)[:-1]
    
    weights +=sleigh_weight
    weights = np.append(weights,sleigh_weight)
    weights = np.cumsum(weights[::-1])[::-1] # flip, cummulative sum, flip again
    
    weighted_dist = np.sum(weights*distances)
    print("weighted_dist ",weighted_dist)
    return weighted_dist


In [ ]:
a = np.array([11]*10)
w = 10

weights = np.append(a,w)
weights = np.cumsum(weights[::-1])[::-1]
weights

In [ ]:
    
weight_limit = 1000
sleigh_weight = 10

entries = df.head(10)
trip1 = entries[:5].copy()
trip2 = entries[5:].copy()

trips = [trip1.copy(),trip2.copy()]

WRW = weighted_reindeer_weariness(trips)
print("Total Wariness: ", WRW)


In [ ]:
[[  1.57079633   0.        ]
 [ 16.34576887   6.30354513]
 [ 12.49474931  28.62639556]
 [ 27.79461514  60.03249479]
 [ 44.42699238 110.11421611]
 [-69.85408841  87.94687788]
 [  1.57079633   0.        ]]
weighted_dist  4998503.445827983
[[  1.57079633   0.        ]
 [ 53.56796981 -71.35930809]
 [ 12.90258404  79.96694891]
 [ -6.29109889 -64.89175089]
 [ -2.68531605 111.08975819]
 [ 38.42886187 101.97367095]
 [  1.57079633   0.        ]]
weighted_dist  6032121.714568638
Total Wariness:  11030625.16039662

# Graph Implementation
 This is a naive implementation of a graph assigning each of the n coordinate for a gift delivery as a unique tour id.
- obviously it results in n tours which is far from optimal
- the TourId is stored as a new column in the dataframe
- I guess this could be a starting point for building a solution

In [ ]:
class Graph:
    def __init__(self, gifts,tourId_provided = False):
        self.numEdges = 0
        self.sort_gifts = False
        #data = self.init_tours(gifts)
        #self.tourgraph = pd.DataFrame(data)
        if tourId_provided:
            """
            Assuming that an initial list of gifts with 
            corresponding tourIds is available.
            """
            self.tourgraph = gifts
            self.sort_gifts = True
        else:
            """
            assuming that just the original df is given 
            --> Naive tourlist with n tours
            """
            data = self.init_tours(gifts)
            self.tourgraph = pd.DataFrame(data)
        
    def init_tours(self, gifts):
        tripIds = np.arange(len(gifts))
        gifts_copy = gifts.copy()
        gifts_copy['TripId'] = tripIds
        return gifts_copy
    
    def weighted_reindeer_weariness(self):
        weighted_weariness = 0
        tot_distance = 0
        trips = self.tourgraph
        grouped_trips = trips.groupby('TripId')
      
        for group_name, trip in grouped_trips:
            if self.sort_gifts:
                trip = trip.sort_values(['Latitude','Longitude'],ascending=False)
            trip
            weights = trip['Weight'].to_numpy()
            coordinates = trip[['Latitude','Longitude']].to_numpy()
            current_wearniess= self.weighted_distance(coordinates,weights)
            weighted_weariness += current_wearniess 

        #print("weighted_weariness ",weighted_weariness)
        return weighted_weariness
    
    def weighted_distance(self, coordinates, weights,weight_limit=1000,sleigh_weight=10):
        startweight = sleigh_weight + np.sum(weights)
        if startweight > weight_limit:
            return -1

        north_pole = np.radians([90,0])
        coords = np.vstack((north_pole,np.radians(coordinates),north_pole))
        print("coords (function)",coords)
        distances = []
        for i in range(len(coords)-1):
            distances.append(haversine_distances([coords[i],coords[i+1]])[0][1]*6371000/1000)
            
        distances = np.array(distances)
        print("distances (function):",distances)
        #adj_matrix = haversine_distances(coords,np.roll(coords.copy(),-1,axis=0))
        #adj_matrix = adj_matrix * 6371 #6371000/1000
        #distances = np.diag(adj_matrix)[:-1]

        #weights +=sleigh_weight
        weights = np.append(weights,sleigh_weight)
        weights = np.cumsum(weights[::-1])[::-1] # flip, cummulative sum, flip again
        weighted_dist = np.sum(weights*distances)
        
        return weighted_dist

    
#graph = Graph(df.head(10))
#tours = graph.tourgraph      

In [ ]:
df = pd.read_csv('../data/gifts.csv')
graph = Graph(df)
weariness = graph.weighted_reindeer_weariness()
weariness

Output with n (number of gifts) tours: 

44314259640.59521

In [ ]:

graph.weighted_reindeer_weariness(tours)


In [ ]:
1/4314259640.59521 * 4608298.715468244

# Clustering

In [ ]:
df = pd.read_csv('../data/gifts.csv')
offset = 0.5
slices = []
for i in range(0,360):
    j = i+0.5
    slices.append(df[(df['Longitude']>(j-offset)) & (df['Longitude']<(j+offset))])
    
for s in slices[:5]:
    s.plot.scatter('Longitude','Latitude')
    s.describe()

first_slice = slices[0]

In [ ]:
from sklearn.cluster import KMeans
import seaborn as sns
import numpy as np
X = first_slice[['Latitude','Longitude']]
kmeans = KMeans(n_clusters=10, random_state=0).fit(X)
labels = kmeans.labels_
first_slice['cluster_labels'] = kmeans.labels_

centers = kmeans.cluster_centers_

groups = first_slice.groupby('cluster_labels')
for group_name, group in groups:
    sum_weight = group['Weight'].sum()
    print("sum_weight ", sum_weight)



s.plot.scatter('Longitude','Latitude')
s.describe()
fig, ax = plt.subplots(figsize=(20,8))
sns.scatterplot(data=first_slice, x="Longitude", y="Latitude", hue='cluster_labels', size=2)



# Slicing and initial Tours

In [ ]:
import functools as ft
from collections import ChainMap

def create_sclices(df):
    slices = []
    offset=0.5
    for i in range(-180,181):
        j = i+offset
        slices.append(df[(df['Longitude']>=(j-offset)) & (df['Longitude']<(j+offset))])
    return slices

def convert_tour_dict_to_df(tours):
    res = {} 
    for dict in tours: 
        for list in dict: 
            if list in res: 
                res[list] += (dict[list]) 
            else: 
                res[list] = dict[list] 
    return pd.DataFrame(res)

def append_location(tour,row,tripId):
        tour['GiftId'].append(row['GiftId'])
        tour['Latitude'].append(row['Latitude'])
        tour['Longitude'].append(row['Longitude'])
        tour['Weight'].append(row['Weight']) 
        tour['TripId'].append(tripId) 
        
def create_tours_from_slices(slices_list):
    slices = slices_list
    tours = []
    tripId = 0
    
    tour = {"GiftId" : [],"Latitude" : [],"Longitude" :[],"Weight" : [],"TripId":[]}

    for s in slices:
        for index,row in s.iterrows():
            sum_current_tour = ft.reduce(lambda x,y:x+y,tour['Weight'],0) 
            if (sum_current_tour+row['Weight'])<=weight_limit:
                append_location(tour,row,tripId)
            else:
                tripId +=1
                sum_current_tour = 0
                tours.append(tour.copy())
                tour = {"GiftId" : [],"Latitude" : [],"Longitude" :[],"Weight" : [],"TripId":[]}
                append_location(tour,row,tripId)
                
    tours.append(tour.copy())
    tours = convert_tour_dict_to_df(tours)
    return tours

In [ ]:
graph = Graph(tours,tourId_provided = True)  
weariness = graph.weighted_reindeer_weariness()

print(weariness)
print(100/44314259640.59521*weariness)

In [ ]:
example_tour

# Compare Results
1. Output with n tours
2. Output using 1 degree sclices

In [ ]:
from sklearn.metrics.pairwise import haversine_distances
num_elements = 3
example_tour = df.head(3).copy()
example_tour['TripId']=[0,0,0]
example_tour = example_tour.sort_values(['Latitude','Longitude'],ascending=False)


north_pole = np.radians(np.array([90,0]))
loc_a = np.radians(np.array([example_tour.iloc[0]['Latitude'],example_tour.iloc[0]['Longitude']]))
loc_b = np.radians(np.array([example_tour.iloc[1]['Latitude'],example_tour.iloc[1]['Longitude']]))
loc_c = np.radians(np.array([example_tour.iloc[2]['Latitude'],example_tour.iloc[2]['Longitude']]))



distances = []

distance_1 = haversine_distances([north_pole,loc_a])[0][1]*6371000/1000
distance_2 = haversine_distances([loc_a,loc_b])[0][1]*6371000/1000
distance_3 = haversine_distances([loc_b,loc_c])[0][1]*6371000/1000
distance_4 = haversine_distances([loc_c,north_pole])[0][1]*6371000/1000


print(distance_1,distance_2,distance_3,distance_4)

weight_1 = np.sum(example_tour['Weight'])+10
weight_2 = weight_1 - example_tour.iloc[0]['Weight']
weight_3 = weight_2 - example_tour.iloc[1]['Weight']
weight_4 = 10
weights = np.array([weight_1,weight_2,weight_3,weight_4])
print("weights 1", weights)

weariness = distance_1*weight_1+distance_2*weight_2+distance_3*weight_3+distance_4*weight_4
print("wearness 1 : ",weariness)



graph = Graph(example_tour,tourId_provided = True)  
weariness_n = graph.weighted_reindeer_weariness()
print("wearness 2 : ",weariness_n)

In [ ]:
#df = pd.read_csv('../data/gifts.csv')
#graph = Graph(df,tourId_provided = False)  
#weariness_n = graph.weighted_reindeer_weariness()
print("1. output with n tours", weariness_n)

df = pd.read_csv('../data/gifts.csv')
slices_list = create_sclices(df)
tours = create_tours_from_slices(slices_list)
graph = Graph(tours,tourId_provided = True)  
weariness_slices = graph.weighted_reindeer_weariness()
print("2. Output using 1 degree sclices : ", weariness_slices)

print("  ")
print("weariness difference : ", weariness_n-weariness_slices)


In [ ]:
print("weariness difference : ", weariness_n-weariness_slices)
print(1/weariness_n*weariness_slices)
best_result = 12384507107.54550

print("difference best result: ",best_result-weariness_slices)
print(1/best_result*weariness_slices)

In [ ]:
len(tours)

In [ ]:
tours